In [5]:
import gc
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

In [6]:
path = '../datasets/B101/'
files = os.listdir(path)
files = [file for file in files if file.endswith('_trimmed.mat')]

In [7]:
dataSet = {}
for file in files:
    fileName = os.path.basename(file)
    fileName = fileName.split('.')[0]
    fileName = fileName.removeprefix('Channel')
    fileName = fileName.removesuffix('_trimmed')
    dataSet[fileName] = sio.loadmat(path + file)
files = None

In [8]:
dataSet.keys()

dict_keys(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '62', '7', '8', '9'])